In [1]:
%pip install git+https://github.com/huggingface/transformers.git


Note: you may need to restart the kernel to use updated packages.Collecting git+https://github.com/huggingface/transformers.git
  Cloning https://github.com/huggingface/transformers.git to c:\users\armaan khetarpaul\appdata\local\temp\pip-req-build-gied177w
  Resolved https://github.com/huggingface/transformers.git to commit f1732e1374a082bf8e43bd0e4aa8a2da21a32a21
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'



  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git 'C:\Users\Armaan Khetarpaul\AppData\Local\Temp\pip-req-build-gied177w'


In [2]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline

In [4]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain.llms import HuggingFacePipeline


In [6]:
from langchain import PromptTemplate, LLMChain


In [8]:
import torch

In [9]:
%pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [10]:
# !apt-get install -y python3-sentencepiece
!git clone https://github.com/google/sentencepiece.git
!cd sentencepiece
!mkdir build
!cd build
!cmake
!make
!make install

Cloning into 'sentencepiece'...
'cmake' is not recognized as an internal or external command,
operable program or batch file.
'make' is not recognized as an internal or external command,
operable program or batch file.
'make' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
from transformers import LlamaTokenizer


In [13]:
tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

C:\Users\Armaan Khetarpaul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Armaan Khetarpaul\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [14]:
%pip install bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [18]:
from accelerate import Accelerator
from transformers import LlamaForCausalLM

accelerator = Accelerator()

base_model = LlamaForCausalLM.from_pretrained(
    "chavinlo/alpaca-native",
)


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1048576 bytes.

In [12]:
! pip install accelerate

In [13]:
pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=500,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)


In [14]:
! pip install xformers


In [34]:
local_llm = HuggingFacePipeline(pipeline=pipe)


In [35]:
from langchain import PromptTemplate, LLMChain


In [36]:


template = """
Write a SQL Query given the table name {Table} and columns as a list {Columns} for the given question : 
{question}.
Winning or victory means position=1.

"""



In [37]:
prompt = PromptTemplate(template=template, input_variables=["Table","question","Columns"])


In [38]:
llm_chain = LLMChain(prompt=prompt, llm=local_llm)


In [39]:
def get_llm_response(tble,question,cols):
    llm_chain = LLMChain(prompt=prompt, 
                         llm=local_llm
                         )
    response= llm_chain.run({"Table" : tble,"question" :question, "Columns" : cols})
    print(response)

In [40]:
tble = "eci_kr_2023"
cols = ['ac_no', 'ac_name_x', 'regions', 'caste', 'locality', 'district', 's_no', 'candidate_name', 'party', 'evm_votes', 'postal_votes', 'votes', 'vote_share', 'st_name', 'st_no', 'ac_name_y', 'constituency_code', 'position', 'total_votes', 'margin', 'margin_percent', 'vote_share_percent', 'margin_buckets_percent', 'margin_buckets_votes']
question="How many MLAs won with more than 1000 votes"


In [41]:
get_llm_response(tble,question,cols)

SELECT * FROM eci_kr_2023 WHERE position = 1 AND total_votes > 1000;


In [85]:
#get_llm_response(tble,question,cols)


SELECT candidate_name, party, SUM(votes) AS totalVotes FROM election1 GROUP BY candidate_name HAVING COUNT (DISTINCT candidate_name) > 1 AND SUM (totalVotes) > 1000;


In [33]:
#new
#get_llm_response(question)

 I cannot answer this question because it requires counting the number of MLA wins which exceed a certain threshold. This type of task is beyond the capability of GPT language models.


In [95]:
conn = sqlite3.connect("data.db")


In [96]:
cursor = conn.cursor()


In [97]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

In [98]:
for table in tables:
    print(table[0])

elec_data
eci_kr_2023


In [62]:
cursor.execute("SELECT * FROM eci_kr_2023 limit 5;")
results = cursor.fetchall()


In [63]:

# Print the retrieved data
for row in results:
    print(row)

(1, 'Nippani', 'Maharashtra - Karnataka', 'Lingayat', 'Rural', 'Belgaum ', 4, 'JOLLE SHASHIKALA ANNASAHEB', 'Bharatiya Janata Party', 72952, 396, 73348, 39.14, 'Karnataka', 'S10', 'Nippani', 1, 1, 187412, 7292.0, 3.890892792, 39.13730177, '(0-5)', 'Between 5000 & 10000')
(1, 'Nippani', 'Maharashtra - Karnataka', 'Lingayat', 'Rural', 'Belgaum ', 5, 'UTTAM RAOSAHEB PATIL', 'Nationalist Congress Party', 65614, 442, 66056, 35.25, 'Karnataka', 'S10', 'Nippani', 1, 2, 187412, 21949.0, 11.71162999, 35.24640898, '(10-20)', '>20000')
(1, 'Nippani', 'Maharashtra - Karnataka', 'Lingayat', 'Rural', 'Belgaum ', 1, 'KAKASAHEB P. PATIL', 'Indian National Congress', 43932, 175, 44107, 23.53, 'Karnataka', 'S10', 'Nippani', 1, 3, 187412, 43188.0, 23.04441551, 23.53477899, '(>20)', '>20000')
(1, 'Nippani', 'Maharashtra - Karnataka', 'Lingayat', 'Rural', 'Belgaum ', 11, 'NOTA', 'None of the Above', 914, 5, 919, 0.49, 'Karnataka', 'S10', 'Nippani', 1, 4, 187412, 273.0, 0.145668367, 0.490363477, '(0-5)', 'L

In [99]:
cursor.close()
conn.close()